In [1]:
import jax.numpy as jnp
import numpy as np
import time
import chex
import jax
import gymnasium as gym
import exciting_environments as excenv
import random
from exciting_environments import PMSM
import matplotlib.pyplot as plt
import diffrax
from jax.tree_util import tree_flatten, tree_unflatten, tree_structure

In [2]:
BATCH_SIZE=5
new_motor_env = PMSM(
    saturated=True,
    batch_size=BATCH_SIZE,
    control_state=["i_d","i_q"])

In [3]:
acts=jnp.full((5,8),jnp.array([1,1,1,1,1,1,1,1])*0.03)
acts=jnp.stack([acts,acts],axis=2)

In [4]:
state=new_motor_env.vmap_init_state()
for _ in range(acts.shape[1]):
    act=jnp.repeat(jnp.array([0.03,0.03])[:,None],BATCH_SIZE,axis=1).T
    obs,state=new_motor_env.vmap_step(state=state,action=act)
    print(obs[0,:])
    state_l=state

[-2.0658602e-04 -4.6228198e-03  1.0000000e+00 -4.1548457e-04
  9.9950653e-01  3.1410761e-02  2.8553503e-02  3.1379886e-02
            nan            nan]
[ 4.9163075e-03 -7.6355794e-03  1.0000000e+00 -6.5322197e-04
  9.9802673e-01  6.2790520e-02  2.8553499e-02  3.1379890e-02
            nan            nan]
[ 9.7074732e-03 -1.0689330e-02  1.0000000e+00 -8.8341557e-04
  9.9556196e-01  9.4108313e-02  2.8553501e-02  3.1379890e-02
            nan            nan]
[ 0.01416366 -0.01378133  1.         -0.00110685  0.9921147   0.12533323
  0.0285535   0.03137989         nan         nan]
[ 0.01828173 -0.0169088   1.         -0.00132437  0.98768836  0.15643448
  0.0285535   0.03137989         nan         nan]
[ 0.02205866 -0.02006889  1.         -0.00153692  0.9822872   0.18738131
  0.0285535   0.03137989         nan         nan]
[ 0.02549149 -0.02325874  1.         -0.00174549  0.97591674  0.21814324
  0.0285535   0.03137989         nan         nan]
[ 0.02857737 -0.02647543  1.         -0.001951

In [5]:
state=new_motor_env.vmap_init_state()
obs,states,last_state=new_motor_env.vmap_sim_ahead(state,
                   acts,
                   new_motor_env.tau,
                   new_motor_env.tau
                   )
obs[0]

Array([[ 0.0000000e+00,  0.0000000e+00,  1.0000000e+00,  0.0000000e+00,
         1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  0.0000000e+00,
                   nan,            nan],
       [-2.0658602e-04, -4.6228198e-03,  1.0000000e+00, -4.1548457e-04,
         9.9950653e-01,  3.1410761e-02,  0.0000000e+00,  0.0000000e+00,
                   nan,            nan],
       [ 4.9163066e-03, -7.6355790e-03,  1.0000000e+00, -6.5322191e-04,
         9.9802673e-01,  6.2790520e-02,  0.0000000e+00,  0.0000000e+00,
                   nan,            nan],
       [ 9.7074723e-03, -1.0689329e-02,  1.0000000e+00, -8.8341546e-04,
         9.9556196e-01,  9.4108313e-02,  0.0000000e+00,  0.0000000e+00,
                   nan,            nan],
       [ 1.4163658e-02, -1.3781331e-02,  1.0000000e+00, -1.1068460e-03,
         9.9211472e-01,  1.2533323e-01,  0.0000000e+00,  0.0000000e+00,
                   nan,            nan],
       [ 1.8281728e-02, -1.6908795e-02,  1.0000000e+00, -1.3243706e-03,
   

- add desired reference in states to generate reward, truncated and terminated

In [6]:
import jax_dataclasses as jdc
init_state=new_motor_env.vmap_init_state()
with jdc.copy_and_mutate(states, validate=False) as states_ref:
            states_ref.reference.i_d=jnp.full((5,10),init_state.physical_state.i_d.reshape(-1,1))
            states_ref.reference.i_q=jnp.full((5,10),init_state.physical_state.i_q.reshape(-1,1))


In [7]:
new_motor_env.vmap_generate_rew_trunc_term_ahead(states_ref,acts)

(Array([[[-0.10037408],
         [-0.38658813],
         [-0.9773286 ],
         [-1.8306293 ],
         [-2.9068542 ],
         [-4.168803  ],
         [-5.5818057 ],
         [-7.113816  ],
         [-8.735495  ]],
 
        [[-0.10037408],
         [-0.38658813],
         [-0.9773286 ],
         [-1.8306293 ],
         [-2.9068542 ],
         [-4.168803  ],
         [-5.5818057 ],
         [-7.113816  ],
         [-8.735495  ]],
 
        [[-0.10037408],
         [-0.38658813],
         [-0.9773286 ],
         [-1.8306293 ],
         [-2.9068542 ],
         [-4.168803  ],
         [-5.5818057 ],
         [-7.113816  ],
         [-8.735495  ]],
 
        [[-0.10037408],
         [-0.38658813],
         [-0.9773286 ],
         [-1.8306293 ],
         [-2.9068542 ],
         [-4.168803  ],
         [-5.5818057 ],
         [-7.113816  ],
         [-8.735495  ]],
 
        [[-0.10037408],
         [-0.38658813],
         [-0.9773286 ],
         [-1.8306293 ],
         [-2.9068542 ],
    